In [ ]:
import jieba
import urllib
import re
import sys
import requests
from PIL import Image, ImageDraw,ImageFont
import numpy as np

In [ ]:
# 找关键词
def getKeyword(text):
    tags = jieba.analyse.extract_tags(text,topK=5)
    print('TF-IDF关键词提取结果：'+'、'.join(tags))
    # tags = jieba.analyse.textrank(text,topK=6)
    # print('TextRank关键词提取结果：'+'、'.join(tags))
    return tags

In [ ]:
# 获得对应图片
def getPage(keyword):
    keyword = urllib.parse.quote(keyword, safe='/')
    url_begin = "http://image.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word="
    url = url_begin+ keyword + "&pn=0&gsm=78&ct=&ic=0&lm=-1&width=0&height=0"
    print(url)
    return url
 
# 找到图片地址
def get_onepage_urls(onepageurl):
    try:
        html = requests.get(onepageurl).text
    except Exception as e:
        print(e)
        pic_urls = []
        return pic_urls
    pic_urls = re.findall('"objURL":"(.*?)",', html, re.S)
    # 取所有图片中的第一张
    print(pic_urls[1])
    return pic_urls[1] 
 
# 下载图片
def getImg(imgurl,name):
    filePath = "images/"
    if not os.path.exists(filePath):
        os.makedirs(filePath)
    try:
        urllib.request.urlretrieve(imgurl,'./images/'+name+'.jpg')
        print('图片 '+name+' 下载完成')
    except Exception as e:
        print('图片 '+name+' 下载失败')



In [ ]:
# 图片手绘风格
def changeStyle(keyword):
    img = 'images/' + keyword + '.jpg'
    a = np.asarray(Image.open(img).convert('L')).astype('float')
    
    depth = 10.                    
    grad = np.gradient(a)                  #梯度值，   
    grad_x, grad_y = grad          
    grad_x = grad_x*depth/100.       #列梯度值*0.1
    grad_y = grad_y*depth/100.
    
    A = np.sqrt(grad_x**2 + grad_y**2 + 1.)  #相当于grad_z=1
    uni_x = grad_x/A
    uni_y = grad_y/A
    uni_z = 1./A          #梯度归一化
    
    vec_el = np.pi/2.2             
    vec_az = np.pi/4.
                    
    dx = np.cos(vec_el)*np.cos(vec_az)
    dy = np.cos(vec_el)*np.sin(vec_az) 
    dz = np.sin(vec_el)             #长度为1，投影x,y,z长度
    
    b = 255*(dx*uni_x + dy*uni_y + dz*uni_z)
    b = b.clip(0,255)
    
    outputPath = './output/'
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
    im = Image.fromarray(b.astype('uint8'))    
    im.save('output/'+keyword+'.jpg')
    print('save image'+keyword)

In [ ]:
# 漫画内容
def generateStoryboard(keywords,title):
    storyboard = Image.new('L',(2500,3500),'white')
    idraw = ImageDraw.Draw(storyboard)

    # add title
    idraw.polygon([(180,125),(1281,125),(1200,1088),(180,1000),(180,125)], fill='black')
    myfont = ImageFont.truetype('font.TTF', 150)
    idraw.text((300,300), title,font=myfont, fill='white')

    # add image
    pboard = storyboard.load()
    for i in range(0,len(keywords)):
        url = 'output/'+keywords[i]+'.jpg'
        im = Image.open(url)
        # change size
        im = im.resize((1300,1300),Image.ANTIALIAS)
        pim = im.load()
        if i==0:
            for i in range(1051):
                for j in range(1032):
                    pboard[i+1299,j+125] = pim[i,j]
            print("finish 0")
        if i==1:
            for i in range(1100):
                for j in range(1251):
                    pboard[i+180,j+1110] = pim[i,j]
            print("finish 1")
        if i==2:
            for i in range(1056):
                for j in range(1068):
                    pboard[i+1294,j+1200] = pim[i,j]
            print("finish 2")
        if i==3:
            for i in range(1103):
                for j in range(1052):
                    pboard[i+180,j+2259] = pim[i,j]
            print("finish 3")
        if i==4:
            for i in range(1050):
                for j in range(1059):
                    pboard[i+1300,j+2252] = pim[i,j]
            print("finish 4")
    
    # add storyboard
    idraw = ImageDraw.Draw(storyboard)
    idraw.polygon((180,1000,1200,1088,1299,1094,2350,1157,2350,1267,1294,1200,1200,1185,180,1110),fill='white')
    idraw.polygon((180,2361,1280,2153,1373,2169,2350,2268,2350,2389,1380,2252,1283,2259,180,2471),fill='white')
    idraw.polygon((1281,125,1383,125,1299,1094,1294,1200,1373,2169,1380,2252,1300,3311,1200,3311,1283,2259,1280,2153,1200,1185,1200,1088),fill='white')

    idraw.line((1383,125,2350,125,2350,1157,1299,1094,1383,125),fill='black',width=10)
    idraw.line((180,1110,1200,1185,1280,2153,180,2361,180,1110),fill='black',width=10)
    idraw.line((1294,1200,2350,1267,2350,2268,1373,2169,1294,1200),fill='black',width=10)
    idraw.line((180,2471,1283,2259,1200,3311,180,3311,180,2471),fill='black',width=10)
    idraw.line((1380,2252,2350,2389,2350,3311,1300,3311,1380,2252),fill='black',width=10)
    storyboard.show()
    storyboard.save('output.jpg')

In [ ]:
if __name__ == '__main__':
    inputword = '昨日，江西九江市永修县三角联圩新建区大塘坪乡防汛责任段溃堤，洪水大量涌入，5.036万亩耕地、2.6万群众受到威胁。江西消防总队迅速调派救援人员，截至今天10时，成功救援被困群众1000人，疏散被困群众2000人。'
    title = '洪水'
    keywords = getKeyword(inputword)
    for keyword in keywords:
        url = getPage(keyword)
        imgURL = get_onepage_urls(url)
        getImg(imgURL,keyword)
        changeStyle(keyword)
    generateStoryboard(keywords,title)
